# Article Spinner

In [42]:
import nltk
#For random number generation and probabilities
import random
import numpy
#For xml parser
from bs4 import BeautifulSoup
from future.utils import iteritems

In [43]:
#Load Positive reviews
positive_reviews = BeautifulSoup(open('electronics/positive.review').read(), "lxml")
positive_reviews = positive_reviews.findAll('review_text')

#Step 1 is to create Trigrams
#Create a dictionary and remember key is previous and next word
trigrams = {}
for review in positive_reviews:
    #Lowercase the reviews
    s = review.text.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    #Only count from zero to the third last word
    for i in range(len(tokens) - 2):
        #Keys are tuples and they are immutable and list cannot
        k = (tokens[i], tokens[i+2])
        if k not in trigrams:
            #Initialize it to an array
            trigrams[k] = []
        trigrams[k].append(tokens[i+1])   
        

In [44]:
#Need to tranform these into a probability vector:
#Create another dictionary:
trigram_probabilities = {}
for k, words in iteritems(trigrams):
    if len(set(words)) > 1:
        #Create another dictionary
        d = {}
        n = 0
        #Use Maximum Likelihood Estimate
        for w in words:
            if w not in d:
                d[w] = 0
            d[w] += 1
            n += 1
        for w,c in iteritems(d):
            #convert these into probabilities
            d[w] = float(c) / n
        trigram_probabilities[k] = d
        
            

In [45]:
#Create a function to randomly sample from these trigram probabilities:
#takes the argument as dictionary:
def random_sample(d):
    r = random.random()
    #Keep a track of the cumulative probability
    cumulative = 0
    for w,p in iteritems(d):
        cumulative += p
        if r < cumulative:
            return w
        
    

In [48]:
#Create a function just to test the spinner:
# It's going to randomly choose a review and try to spin it
def test_spinner():
    review = random.choice(positive_reviews)
    s = review.text.lower()
    print("Original:", s)
    #Now tokenize it
    tokens = nltk.tokenize.word_tokenize(s)
    #Loop through each token and choose with the small probability to replace:
    for i in range(len(tokens) - 2):
        if random.random() < 0.2:
            k = (tokens[i], tokens[i+2])
            if k in trigram_probabilities:
                w = random_sample(trigram_probabilities[k])
                tokens[i+1] = w
    print("Spun:")  
    #Replace as nltk tries to tokenize punctuation as well
    print(" ".join(tokens).replace(" .",".").replace(" '","'").replace(" ,",",").replace("$ ","$").replace(" !","!"))
    
    
    
            
    

In [51]:
test_spinner()

Original: 
i was surprised at the sound quality of this small speaker system.  i purchased it to use in my bedroom on my nightstand and am very pleased.  you can hear the music in several other rooms if the volume is turned up, without distortion.

i am very pleased with my purchase and would recommend the speaker system to others

Spun:
i was surprised at the sounds quality of this small speaker system. i purchased it to use in my bedroom on my nightstand and am very pleased. you can hear the music in several other rooms if the volume is turned up, without problem. i am very happy with my purchase and would recommend the speaker system to others


 www.udemy.com/data-science-logistic-regression-in-python

www.udemy.com/data-science-deep-learning-in-python